In [9]:
# basics
import numpy as np
import pandas as pd
import datetime
import re
import os, glob
import pickle
import textwrap

# statistics
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.model_selection import KFold
from datetime import date, timedelta
import tqdm


# plotting
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = [15,12]
from matplotlib.ticker import MultipleLocator

In [10]:
data = pd.read_csv('Data/statsmodels.csv', sep=';', index_col='Date', parse_dates=['Date']).drop(['Unnamed: 0'], axis=1)
display(data.tail())

main = pd.read_csv('Data/statsmodels rel.csv', sep=';', index_col='Date', parse_dates=['Date']).drop(['Unnamed: 0'], axis=1)
display(main.tail())

,No_cases,Rolling_R,Workplace,Events,Daycare,Higher Education,Secondary Education,Primary Education,Restaurants/Cafes,Transport,...,Masks,MassGather50,MassGatherAll,Shops,Outdoor,Religion,Gatherings,Stayhome,Telework,Prikken
Date,,,,,,,,,,,,,,,,,,,,,
2021-05-02,5753,0.907408,2.0,2.0,0.0,2.0,1.0,0.0,2.0,0.0,...,2.0,2.0,2.0,1.0,0.0,0.0,2.0,3.0,2.0,5651041.0
2021-05-03,5716,0.893238,2.0,2.0,0.0,2.0,1.0,0.0,2.0,0.0,...,2.0,2.0,2.0,1.0,0.0,0.0,2.0,3.0,2.0,5757731.0
2021-05-04,5336,0.880369,2.0,2.0,0.0,2.0,1.0,0.0,2.0,0.0,...,2.0,2.0,2.0,1.0,0.0,0.0,2.0,3.0,2.0,5860446.0
2021-05-05,5936,0.869616,2.0,2.0,0.0,2.0,1.0,0.0,2.0,0.0,...,2.0,2.0,2.0,1.0,0.0,0.0,2.0,3.0,2.0,5980676.0
2021-05-06,5365,0.860512,2.0,2.0,0.0,2.0,1.0,0.0,2.0,0.0,...,2.0,2.0,2.0,1.0,0.0,0.0,2.0,3.0,2.0,6101271.0


,No_cases,Workplace,Events,Daycare,Higher Education,Secondary Education,Primary Education,Restaurants/Cafes,Transport,Entertainment,...,MassGather50,MassGatherAll,Shops,Outdoor,Religion,Gatherings,Stayhome,Telework,Prikken,Rolling_R
Date,,,,,,,,,,,,,,,,,,,,,
2021-05-02,0.457531,1.0,1.0,0.0,1.0,0.5,0.0,1.0,0.0,1.0,...,1.0,1.0,0.5,0.0,0.0,1.0,0.75,1.0,0.926207,0.907408
2021-05-03,0.454589,1.0,1.0,0.0,1.0,0.5,0.0,1.0,0.0,1.0,...,1.0,1.0,0.5,0.0,0.0,1.0,0.75,1.0,0.943694,0.893238
2021-05-04,0.424368,1.0,1.0,0.0,1.0,0.5,0.0,1.0,0.0,1.0,...,1.0,1.0,0.5,0.0,0.0,1.0,0.75,1.0,0.960529,0.880369
2021-05-05,0.472085,1.0,1.0,0.0,1.0,0.5,0.0,1.0,0.0,1.0,...,1.0,1.0,0.5,0.0,0.0,1.0,0.75,1.0,0.980234,0.869616
2021-05-06,0.426674,1.0,1.0,0.0,1.0,0.5,0.0,1.0,0.0,1.0,...,1.0,1.0,0.5,0.0,0.0,1.0,0.75,1.0,1.000000,0.860512


In [21]:
len(main)

437

In [11]:
def shift_outcome(X,y,s,shift_date="y"):

    # If shift_date="y", then the date of y is shifted s days into the past. If shift_date="X", then the date of X is shifted s days into the future.

        if shift_date=="y":
            
            y_shifted = y.reset_index().copy()
            y_shifted["Date"] = y_shifted[["Date"]].applymap(lambda x: x.date()-timedelta(s))
            y_shifted = y_shifted.set_index(["Date"])
            y_shifted.drop(y_shifted.loc[np.isnan(y_shifted.values)].index,inplace=True)
            data2 = X.join(y_shifted,how="inner")
            X = data2[X.columns]
            y_shifted = data2[y.name]
            
            return X,y_shifted
        
        if shift_date=="X":
        
            X_shifted = X.reset_index().copy()
            X_shifted["Date"] = X_shifted[["Date"]].applymap(lambda x: x.date()+timedelta(s))
            X_shifted = X_shifted.set_index(["Date"])
            y.drop(y.loc[np.isnan(y.values)].index,inplace=True)
            data = X_shifted.join(y,how="inner")
            X_shifted = data[X.columns]
            y = data[y.name]
            
            return X_shifted,y

In [12]:
%%time
shiftdays = range(0, 21) 
alphalist = np.power(10,np.linspace(-4,-1,num=17)) 
crossvalcount = 10

# shiftdays = [1, 2, 6]
# alphalist = [0.01, 0.5, 1e-3]


X = main.drop(['No_cases', 'Rolling_R'], axis=1)
Y = main['Rolling_R']

curResDF = pd.DataFrame()
n = 0
totaln = len(shiftdays) * len(alphalist)
for s in shiftdays:
    
    X_used, Y_used = shift_outcome(X,Y,s,shift_date="y")
    
    X_used.columns = [col.replace(' ', '_').replace('/', '_') for col in X_used.columns.values]
    combined = X_used.copy()
    combined['Rolling_R'] = Y_used
    
    formula = 'Rolling_R ~ ' + ' + '.join(X_used.columns)
    
    for a in alphalist:
        print(f'Huidige progressie: {(n / totaln)*100:.2f}% \n shiftday: {s}, alpha: {a}')
        kf = KFold(n_splits=crossvalcount, shuffle=True)

        for train_ind, test_ind in kf.split(X_used):
            #print(train_ind, test_ind)
            train = combined.iloc[train_ind]
            test = combined.iloc[test_ind]
#           
            trainmodel    = smf.ols(formula = formula, data = train)
            testmodel     = smf.ols(formula = formula, data = test)
        
            trainresults  = trainmodel.fit(alpha = a, L1_wt = 1)

            test_params   = []
            for paramname in testmodel.exog_names:
                if paramname in trainresults.params.keys():
                    test_params.append(trainresults.params[paramname])
                else:
                    test_params.append(0)

            obs_train     = np.array(trainmodel.endog)
            obs_test      = np.array(testmodel.endog)
            pred_train    = trainmodel.predict(trainresults.params)
            pred_test     = testmodel.predict(test_params)
            
            result_dict                          = {'shiftdays': s, 'alpha': a}
            result_dict['Test Sample Size']      = len(test)
            result_dict['Training Sample Size']  = len(train)
            result_dict['Loglike Training']      = trainmodel.loglike(trainresults.params)
            result_dict['Loglike Test']          = testmodel.loglike(np.array(test_params))
            result_dict['RSS Training']          = np.sum((obs_train - pred_train)**2)
            result_dict['RSS Test']              = np.sum((obs_test - pred_test)**2)
            result_dict['NVar Training']         = np.sum((obs_train - np.mean(obs_train))**2)
            result_dict['NVar Test']             = np.sum((obs_test - np.mean(obs_test))**2)
            result_dict['R2 Training']           = 1 - result_dict['RSS Training']/result_dict['NVar Training']
            result_dict['R2 Test']               = 1 - result_dict['RSS Test']/result_dict['NVar Test']

            result_dict.update({k:v for k,v in trainresults.params.items()})
            new_DF = pd.DataFrame({k:np.array([v]) for k,v in result_dict.items()})
            curResDF = curResDF.append(new_DF, ignore_index=False)
        n += 1

Huidige progressie: 0.00% 
 shiftday: 0, alpha: 0.0001
Huidige progressie: 0.28% 
 shiftday: 0, alpha: 0.0001539926526059492
Huidige progressie: 0.56% 
 shiftday: 0, alpha: 0.00023713737056616554
Huidige progressie: 0.84% 
 shiftday: 0, alpha: 0.0003651741272548377
Huidige progressie: 1.12% 
 shiftday: 0, alpha: 0.0005623413251903491
Huidige progressie: 1.40% 
 shiftday: 0, alpha: 0.0008659643233600654
Huidige progressie: 1.68% 
 shiftday: 0, alpha: 0.001333521432163324
Huidige progressie: 1.96% 
 shiftday: 0, alpha: 0.002053525026457146
Huidige progressie: 2.24% 
 shiftday: 0, alpha: 0.0031622776601683794
Huidige progressie: 2.52% 
 shiftday: 0, alpha: 0.004869675251658631
Huidige progressie: 2.80% 
 shiftday: 0, alpha: 0.007498942093324558
Huidige progressie: 3.08% 
 shiftday: 0, alpha: 0.011547819846894581
Huidige progressie: 3.36% 
 shiftday: 0, alpha: 0.01778279410038923
Huidige progressie: 3.64% 
 shiftday: 0, alpha: 0.027384196342643614
Huidige progressie: 3.92% 
 shiftday: 0, a

Huidige progressie: 33.89% 
 shiftday: 7, alpha: 0.00023713737056616554
Huidige progressie: 34.17% 
 shiftday: 7, alpha: 0.0003651741272548377
Huidige progressie: 34.45% 
 shiftday: 7, alpha: 0.0005623413251903491
Huidige progressie: 34.73% 
 shiftday: 7, alpha: 0.0008659643233600654
Huidige progressie: 35.01% 
 shiftday: 7, alpha: 0.001333521432163324
Huidige progressie: 35.29% 
 shiftday: 7, alpha: 0.002053525026457146
Huidige progressie: 35.57% 
 shiftday: 7, alpha: 0.0031622776601683794
Huidige progressie: 35.85% 
 shiftday: 7, alpha: 0.004869675251658631
Huidige progressie: 36.13% 
 shiftday: 7, alpha: 0.007498942093324558
Huidige progressie: 36.41% 
 shiftday: 7, alpha: 0.011547819846894581
Huidige progressie: 36.69% 
 shiftday: 7, alpha: 0.01778279410038923
Huidige progressie: 36.97% 
 shiftday: 7, alpha: 0.027384196342643614
Huidige progressie: 37.25% 
 shiftday: 7, alpha: 0.042169650342858224
Huidige progressie: 37.54% 
 shiftday: 7, alpha: 0.06493816315762113
Huidige progress

Huidige progressie: 67.23% 
 shiftday: 14, alpha: 0.00023713737056616554
Huidige progressie: 67.51% 
 shiftday: 14, alpha: 0.0003651741272548377
Huidige progressie: 67.79% 
 shiftday: 14, alpha: 0.0005623413251903491
Huidige progressie: 68.07% 
 shiftday: 14, alpha: 0.0008659643233600654
Huidige progressie: 68.35% 
 shiftday: 14, alpha: 0.001333521432163324
Huidige progressie: 68.63% 
 shiftday: 14, alpha: 0.002053525026457146
Huidige progressie: 68.91% 
 shiftday: 14, alpha: 0.0031622776601683794
Huidige progressie: 69.19% 
 shiftday: 14, alpha: 0.004869675251658631
Huidige progressie: 69.47% 
 shiftday: 14, alpha: 0.007498942093324558
Huidige progressie: 69.75% 
 shiftday: 14, alpha: 0.011547819846894581
Huidige progressie: 70.03% 
 shiftday: 14, alpha: 0.01778279410038923
Huidige progressie: 70.31% 
 shiftday: 14, alpha: 0.027384196342643614
Huidige progressie: 70.59% 
 shiftday: 14, alpha: 0.042169650342858224
Huidige progressie: 70.87% 
 shiftday: 14, alpha: 0.06493816315762113
Hu

In [13]:
CVresults = curResDF.copy()
CVresults['alpha'] = CVresults['alpha'].map('{:.6e}'.format)
CVresults =  CVresults.groupby(['shiftdays','alpha'], as_index = False).agg(
            { 'Loglike Test':['mean','std'],
              'Loglike Training':['mean','std'],
              'R2 Test': ['mean','std'],
              'R2 Training': ['mean','std'],
              'RSS Training' : ['sum'],
              'RSS Test': ['sum'],
              'NVar Training':['sum'],
              'NVar Test':['sum'],
              'Test Sample Size':['sum'],
              'Training Sample Size':['sum']
            })
CVresults.columns = [ 'shiftdays','alpha',
                      'Loglike Test Avgd','Loglike Test Avgd Std',
                      'Loglike Training Avgd','Loglike Training Avgd Std',
                      'R2 Test Avgd','R2 Test Avgd Std',
                      'R2 Training Avgd','R2 Training Avgd Std',
                      'RSS Training Sum',
                      'RSS Test Sum',
                      'NVar Training Sum',
                      'NVar Test Sum',
                      'Test Sample Size',
                      'Training Sample Size'
                    ]
CVresults['RSS per datapoint Training'] = CVresults['RSS Training Sum']/CVresults['Training Sample Size']
CVresults['RSS per datapoint Test']     = CVresults['RSS Test Sum']/CVresults['Test Sample Size']

CVresults['R2 Training Weighted']       = 1 - CVresults['RSS Training Sum']/CVresults['NVar Training Sum']
CVresults['R2 Test Weighted']           = 1 - CVresults['RSS Test Sum']/CVresults['NVar Test Sum']

CVresults['alpha']                      = CVresults['alpha'].astype(np.float64) # return to numbers

CVresults.sort_values(by = ['shiftdays','alpha'], inplace = True)

In [14]:
# manually select best
CVresults.sort_values(by='R2 Test Weighted', ascending=False)

,shiftdays,alpha,Loglike Test Avgd,Loglike Test Avgd Std,Loglike Training Avgd,Loglike Training Avgd Std,R2 Test Avgd,R2 Test Avgd Std,R2 Training Avgd,R2 Training Avgd Std,RSS Training Sum,RSS Test Sum,NVar Training Sum,NVar Test Sum,Test Sample Size,Training Sample Size,RSS per datapoint Training,RSS per datapoint Test,R2 Training Weighted,R2 Test Weighted
4,0,0.000154,26.175281,9.244641,239.559352,11.925386,0.691416,0.113125,0.759970,0.006995,68.219027,8.483928,284.342484,30.865025,437,3933,0.017345,0.019414,0.760081,0.725128
21,1,0.000154,29.469473,10.797431,258.090879,10.719222,0.648361,0.206229,0.748545,0.017614,61.735560,7.462080,246.162408,26.650921,436,3924,0.015733,0.017115,0.749208,0.720007
24,1,0.000237,28.914073,10.977078,257.760968,9.964305,0.682086,0.146178,0.748647,0.010728,61.829741,7.626251,246.206456,27.041004,436,3924,0.015757,0.017491,0.748870,0.717975
18,1,0.000100,30.196582,12.342411,261.624065,18.642489,0.704178,0.099078,0.753088,0.008796,60.779965,7.656649,246.201345,26.993842,436,3924,0.015489,0.017561,0.753129,0.716356
7,0,0.000237,25.932555,11.196773,235.531257,17.463438,0.663244,0.151457,0.753776,0.025217,69.769782,8.829327,284.347426,30.911560,437,3933,0.017740,0.020204,0.754632,0.714368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,16,0.100000,11.445299,5.961688,96.981977,3.756367,-0.439547,0.251134,-0.398714,0.011091,132.985791,14.832893,95.095791,10.345429,421,3789,0.035098,0.035233,-0.398440,-0.433763
289,17,0.100000,11.547487,5.244144,99.404561,2.797669,-0.431646,0.186013,-0.406564,0.008502,130.810619,14.578915,93.010619,10.156904,420,3780,0.034606,0.034712,-0.406405,-0.435370
323,19,0.100000,11.923889,5.736718,101.434090,3.802260,-0.442859,0.168401,-0.414363,0.011962,128.478026,14.315107,90.858026,9.937260,418,3762,0.034152,0.034247,-0.414053,-0.440549
306,18,0.100000,11.872644,6.105225,100.815827,3.465550,-0.449283,0.229080,-0.411654,0.010562,129.370239,14.426921,91.660239,9.976748,419,3771,0.034307,0.034432,-0.411411,-0.446054


In [37]:
shiftdays = 0
alpha = 0.0001539926526059492
crossvalcount = 10

DF = pd.DataFrame()

X = main.drop(['Rolling_R', 'No_cases'], axis=1)
Y = main['Rolling_R']

X_used, Y_used = shift_outcome(X,Y,shiftdays,shift_date="y")
X_used.columns = [col.replace(' ', '_').replace('/', '_') for col in X_used.columns.values]
combined = X_used.copy()
combined['Rolling_R'] = Y_used
formula = 'Rolling_R ~ ' + ' + '.join(X_used.columns)
print(formula)
        
kf = KFold(n_splits=crossvalcount, shuffle=True)

for train_ind, test_ind in kf.split(X_used):
#   print(train_ind, test_ind)
    train = combined.iloc[train_ind]
    test = combined.iloc[test_ind]
       
    trainmodel    = smf.ols(formula = formula, data = train)
    testmodel     = smf.ols(formula = formula, data = test)

    trainresults  = trainmodel.fit(alpha = alpha, L1_wt = 1)#lasso
    print(trainresults.df_resid)
    
#     A = np.identity(len(trainresults.params))
#     A = A[1:,:]
#     print(trainresults.f_test(A))
    
    test_params   = []
    for paramname in testmodel.exog_names:
        if paramname in trainresults.params.keys():
            test_params.append(trainresults.params[paramname])
        else:
            test_params.append(0)

    obs_train     = np.array(trainmodel.endog)
    obs_test      = np.array(testmodel.endog)
    pred_train    = trainmodel.predict(trainresults.params)
    pred_test     = testmodel.predict(test_params)

    result_dict                          = {'shiftdays': shiftdays, 'alpha': alpha}
    result_dict['Test Sample Size']      = len(test)
    result_dict['Training Sample Size']  = len(train)
    result_dict['Loglike Training']      = trainmodel.loglike(trainresults.params)
    result_dict['Loglike Test']          = testmodel.loglike(np.array(test_params))
    result_dict['RSS Training']          = np.sum((obs_train - pred_train)**2)
    result_dict['RSS Test']              = np.sum((obs_test - pred_test)**2)
    result_dict['NVar Training']         = np.sum((obs_train - np.mean(obs_train))**2)
    result_dict['NVar Test']             = np.sum((obs_test - np.mean(obs_test))**2)
    result_dict['R2 Training']           = 1 - result_dict['RSS Training']/result_dict['NVar Training']
    result_dict['R2 Test']               = 1 - result_dict['RSS Test']/result_dict['NVar Test']
    result_dict['F value']               = trainresults.fvalue
    result_dict['P value']               = trainresults.f_pvalue
    
    result_dict.update({k:v for k,v in trainresults.params.items()})
    new_DF = pd.DataFrame({k:np.array([v]) for k,v in result_dict.items()})
    DF = DF.append(new_DF, ignore_index=False)
    
# formula = 'Rolling_R ~ ' + ' + '.join(X_used.columns)  
# finalmodel = smf.ols(data = combined, formula = formula)
# finalresults = finalmodel.fit_regularized(alpha=alpha, L1_wt = 1) #lasso
DF

Rolling_R ~ Workplace + Events + Daycare + Higher_Education + Secondary_Education + Primary_Education + Restaurants_Cafes + Transport + Entertainment + Sports + Indoor + Masks + MassGather50 + MassGatherAll + Shops + Outdoor + Religion + Gatherings + Stayhome + Telework + Prikken
373.0
373.0
373.0
373.0
374.0
373.0
373.0
374.0
374.0
374.0


,shiftdays,alpha,Test Sample Size,Training Sample Size,Loglike Training,Loglike Test,RSS Training,RSS Test,NVar Training,NVar Test,...,Masks,MassGather50,MassGatherAll,Shops,Outdoor,Religion,Gatherings,Stayhome,Telework,Prikken
0,0,0.000154,44,393,268.691083,52.399030,5.862334,0.238005,30.363925,1.152604,...,-0.265428,0.032106,-0.633224,-0.108681,0.259932,-0.125860,0.167202,0.076712,-0.281738,-0.160292
0,0,0.000154,44,393,296.059171,19.800384,5.100151,1.047383,27.017073,4.583739,...,-0.297255,0.060255,-0.524285,-0.120519,0.235911,-0.138019,0.108827,0.086184,-0.191124,-0.164485
0,0,0.000154,44,393,277.892269,35.587069,5.594156,0.511053,30.273680,1.328176,...,-0.273750,0.027642,-0.633857,-0.107159,0.265368,-0.116650,0.172584,0.091662,-0.264421,-0.167419
0,0,0.000154,44,393,277.916640,34.042229,5.593462,0.548229,25.353422,5.964427,...,-0.260949,-0.010722,-0.776307,-0.140755,0.295122,-0.180525,0.278093,0.111459,-0.474371,-0.194222
0,0,0.000154,44,393,289.481397,22.924620,5.273766,0.908722,24.882920,6.254986,...,-0.250032,0.092821,-0.442448,-0.118351,0.195819,-0.068959,0.013624,0.097087,0.027006,-0.156112
0,0,0.000154,44,393,269.058103,51.639825,5.851395,0.246362,30.698465,0.811799,...,-0.275082,0.016934,-0.645294,-0.125287,0.264871,-0.162988,0.170060,0.093658,-0.309812,-0.154546
0,0,0.000154,44,393,325.168430,8.500146,4.397919,1.750571,25.247207,6.250049,...,-0.266606,0.015210,-0.583586,0.022267,0.248626,-0.003790,0.152844,0.091369,-0.270786,-0.168757
0,0,0.000154,43,394,278.342796,35.020141,5.615700,0.493851,30.127950,1.187787,...,-0.252549,0.015587,-0.652034,-0.128760,0.261285,-0.184858,0.168679,0.110548,-0.336149,-0.144485
0,0,0.000154,43,394,284.432561,27.546317,5.444760,0.699142,29.636700,1.941202,...,-0.265024,0.015274,-0.680625,-0.153823,0.273989,-0.180286,0.192876,0.094389,-0.354018,-0.158217
0,0,0.000154,43,394,270.299701,49.611002,5.849722,0.250533,30.677333,0.814226,...,-0.247151,0.011662,-0.648225,-0.105162,0.260140,-0.153152,0.175376,0.099747,-0.328629,-0.143437


In [38]:
pd.options.display.float_format = "{:,.6f}".format
relatief = DF.fillna(0)#.divide(DF.Intercept, axis=0)
relatief = DF.quantile([0.5, .025, 0.975]).T
relatief.columns = ['median', 'low', 'high']
relatief.sort_values(by = ['median','high','low'], inplace = True)
relatief = relatief.drop(['shiftdays', 'alpha', 'Test Sample Size', 'Training Sample Size', 'Loglike Training', 'Loglike Test', 
              'RSS Training', 'RSS Test', 'NVar Training', 'NVar Test', 'R2 Training', 'R2 Test', 'F value', 'P value'])

In [40]:
relatief.reset_index().rename({'index':'Maatregelgroep'}, axis=1).to_csv('LASSO coefficient rel new.csv', sep=';', index=False)

In [39]:
relatief

,median,low,high
MassGatherAll,-0.639575,-0.754778,-0.460861
Workplace,-0.524242,-0.562500,-0.493870
Restaurants_Cafes,-0.352473,-0.567795,-0.267442
Telework,-0.295775,-0.447292,-0.022073
Masks,-0.265226,-0.292266,-0.247799
Transport,-0.200571,-0.252349,-0.144807
Prikken,-0.159254,-0.188492,-0.143673
Religion,-0.145585,-0.183883,-0.018453
Daycare,-0.124421,-0.129945,-0.103607
Shops,-0.119435,-0.150883,-0.006404
